In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split

from sklearn.preprocessing import MinMaxScaler
import time

import random

import torch
import torch.nn as nn

import torch.optim as optim

import copy
import matplotlib.pyplot as plt
import math

import os
os.environ['KMP_DUPLICATE_LIB_OK']='True'



In [ ]:
data = pd.read_csv('station_153211-2022-09_2023-01.csv')


In [ ]:
# Adding / removing columns.
data['DateTime'] = pd.to_datetime(data['created_at'], infer_datetime_format=True)
data.set_index('DateTime', inplace=True)
data['day_of_week'] = data.index.dayofweek
data['month'] = data.index.month
data['is_weekend'] = (data.index.dayofweek >= 5).astype(int)
data['hour_bin'] = data.index.hour // 4 
holidays = pd.to_datetime(['2022-12-31', '2022-12-23'])
data['is_holiday'] = data.index.isin(holidays).astype(int)
data.drop('created_at', axis=1, inplace=True)
data.drop('station_id', axis=1, inplace=True)
data.drop('available_count', axis=1, inplace=True)
data.drop('outlet_count', axis=1, inplace=True)

## KNN

In [ ]:
data = data[~data.index.duplicated(keep='first')]

start_date = data.index[5]
end_date = data.index[-5]
datetime_index = pd.date_range(start=start_date, end=end_date, freq='5T')

data_r = pd.DataFrame(index=datetime_index, columns=data.columns)

k = 10

for target_time in datetime_index:

    data_c = copy.copy(data)
    data_c['time_difference'] = abs(data_c.index - target_time)
    data_c_sorted = data_c.sort_values(by='time_difference')
    k_closest_rows = data_c_sorted.head(k)
    k_closest_rows= k_closest_rows.drop(columns=['time_difference'])
    new_col = k_closest_rows.mean(axis=0)
    data_r.loc[target_time] = new_col

## Normalize

In [ ]:

scaler = MinMaxScaler()

df_normalized = pd.DataFrame(scaler.fit_transform(data_r),
                             index=data_r.index,
                             columns=data_r.columns)

#print(df_normalized['is_holiday'].value_counts())
#(data['is_holiday'].value_counts())


## Split & Dataloaders

In [ ]:
split = int(len(df_normalized) * 0.9)
train_data = df_normalized.iloc[:split]
val_data = df_normalized.iloc[split:]


In [ ]:
from torch.utils.data import Dataset, DataLoader

class datasetMaker(Dataset):
    def __init__(self, data, seq_len=10, future_steps=5):
        # Assuming 'data' is a numpy array or a pandas DataFrame, convert it to a numpy array
        self.data = data.values if isinstance(data, pd.DataFrame) else data
        self.seq_len = seq_len
        self.future_steps = future_steps

    def __len__(self):
        # Subtract seq_len to avoid going out of bounds
        return len(self.data) - self.seq_len - self.future_steps

    def __getitem__(self, index):
        # Get the sequence and label, and convert them to torch tensors
        
        seq = torch.tensor(self.data[index:index+self.seq_len], dtype=torch.float)
        label = torch.tensor(self.data[index+self.seq_len:index+self.seq_len+self.future_steps], dtype=torch.float)
        label=torch.unsqueeze(label[:,1], 1)
        return seq, label
    
future_steps = 36
seq_len = 576
batch_size = 16
        
train_dataset = datasetMaker(train_data, seq_len, future_steps)
train_loader = DataLoader(dataset=train_dataset, batch_size=16, shuffle=True, drop_last=True)
        
val_dataset = datasetMaker(val_data, seq_len, future_steps)
val_loader = DataLoader(dataset=val_dataset, batch_size=16, shuffle=True, drop_last=True)

print(len(val_loader))

for data, label in train_loader:
    print(data.shape, label.shape)
    break


In [ ]:
def train_epoch(epoch, optimizer, loss_function, model, train_loader, future_steps):
    total_loss = 0
    model.train()
    for batch_idx, (data,label) in enumerate(train_loader):
        
        data = data.cuda()
        label = label.cuda()
                
        optimizer.zero_grad()
        
        future_steps = 5
        predictions = model(data, future=future_steps)  
        loss_value = loss_function(predictions,label)
        loss_value.backward()
        optimizer.step()

        total_loss += loss_value.item()
        
    return total_loss / len(train_loader)

def validate_epoch(epoch, loss, model, val_loader, future_steps):
    total_loss = 0
    model.eval()
    with torch.no_grad():
        for batch_idx, (data, label) in enumerate(val_loader):
            
            data = data.cuda()
            label = label.cuda()

            predictions = model(data, future=future_steps)
            loss_value = loss(predictions, label)
            total_loss += loss_value.item()

    return total_loss / len(val_loader)

def a_proper_training(num_epoch, model, optimizer, loss_function, loader, future_steps):
    best_epoch = None
    best_model = None
    best_loss = None
    train_losses = list()
    val_losses = list()
    print("Begin Training")

    for epoch in range(num_epoch):
        start_time = time.time()  # Start time

        train_loss = train_epoch(epoch, optimizer, loss_function, model, train_loader, future_steps)
        val_loss = validate_epoch(0, criterion, model, val_loader, future_steps)
        train_losses.append(train_loss)
        val_losses.append(val_loss)

        if epoch == 0:
            best_loss = val_loss
            
        if val_loss < best_loss:
            best_loss = val_loss
            best_model = copy.deepcopy(model)
            best_epoch = epoch

        end_time = time.time()
        elapsed_time = end_time - start_time
        
        print(f"Epoch {epoch + 1}/{num_epoch}: Train Loss = {train_loss} Val Loss = {val_loss} Elapsed_time = {elapsed_time}")
            
    return (best_model, best_epoch, train_losses, val_losses)


# Transformer Model Info
### * Verkar som att det bara behövs en encoder för vårt task, och ingen decoder
### * Får samma fel som med LSTM... Både bra och dåligt :P
### * Positional encoding ej implementerat ännu

In [ ]:
class SimpleEncoderTransformer(nn.Module):
    def __init__(self, input_size, hidden_layer_size, output_size, seq_length, num_layers=1, dropout=0.0):
        super(SimpleEncoderTransformer, self).__init__()
        
        self.seq_length = seq_length
        self.output_size = output_size 

        self.embedding = nn.Linear(input_size, hidden_layer_size)
        self.PositionalEncoding = PositionalEncoding(d_model=hidden_layer_size, max_len=100)

        encoder_layers = nn.TransformerEncoderLayer(d_model=hidden_layer_size, nhead=1, 
                                                    dim_feedforward=hidden_layer_size, dropout=dropout)
        self.transformer_encoder = nn.TransformerEncoder(encoder_layer=encoder_layers, num_layers=num_layers)

        self.linear = nn.Linear(hidden_layer_size, seq_length * output_size)
        
    def forward(self, x, future=1):
        x = self.embedding(x)
        x = self.PositionalEncoding(x)
        
        output = self.transformer_encoder(x)
        
        output = self.linear(output[:, -1, :])
        output = output.view(-1, self.seq_length, self.output_size) 
        return output

class PositionalEncoding(nn.Module):
    def __init__(self, d_model, max_len=5000):
        super(PositionalEncoding, self).__init__()
        pe = torch.zeros(max_len, d_model)
        position = torch.arange(0, max_len, dtype=torch.float).unsqueeze(1)
        div_term = torch.exp(torch.arange(0, d_model, 2).float() * (-math.log(10000.0) / d_model))
        pe[:, 0::2] = torch.sin(position * div_term)
        pe[:, 1::2] = torch.cos(position * div_term)
        pe = pe.unsqueeze(0).transpose(0, 1)
        self.register_buffer('pe', pe)

    def forward(self, x):
        x = x + self.pe[:x.size(0), :]
        return x
    

In [ ]:
model = SimpleEncoderTransformer(input_size=8, hidden_layer_size=100, output_size=1, seq_length=36, num_layers=5).cuda()

optimizer = optim.Adam(model.parameters(), lr=0.001)
criterion = nn.MSELoss()

best_model, best_epoch, train_losses, val_losses = a_proper_training(
    25, 
    model,
    optimizer,
    criterion,
    train_loader,
    future_steps
)

In [ ]:
plt.plot(train_losses, label="train")
plt.plot(val_losses, label="val")
plt.title("MSE Loss")
plt.legend()

In [ ]:

# Assuming val_loader is your DataLoader and best_model is your model

for data, label in train_loader:
    data = data.cuda()

    # Get predictions from the model
    predictions = best_model(data, future=future_steps)
    
    # Convert tensors to numpy arrays
    predictions = predictions.detach().cpu().numpy()
    labels = label.numpy()
    
    # Determine the number of rows and columns for the grid
    batch_size, sequence_length, _ = predictions.shape
    num_rows = int(math.ceil(batch_size / 4))  # 4 columns for a 4x4 grid
    num_cols = 4
    
    # Create a grid of subplots
    fig, axes = plt.subplots(num_rows, num_cols, figsize=(15, 15))
    fig.suptitle("Predictions vs True Values")
    
    # Plot each sequence in the batch
    for i in range(batch_size):
        row = i // num_cols
        col = i % num_cols
        
        ax = axes[row, col] if num_rows > 1 else axes[col]
        
        # Get the predictions and true values for the current sequence
        t = labels[i, :]
        p = predictions[i, :]
        
        # Plot the predictions and true values on the current subplot
        ax.plot(p, label="Predictions")
        ax.plot(t, label="True Values")
        ax.set_title(f"Sequence {i+1}")
        ax.set_ylim(0, 1)

        ax.legend()
        
        
    
    # Adjust layout and display the plot
    plt.tight_layout()
    plt.show()
    break  # Break after processing the first batch
    
    


In [ ]:

# Assuming val_loader is your DataLoader and best_model is your model

for data, label in val_loader:
    data = data.cuda()

    # Get predictions from the model
    predictions = best_model(data, future=future_steps)
    
    # Convert tensors to numpy arrays
    predictions = predictions.detach().cpu().numpy()
    labels = label.numpy()
    
    # Determine the number of rows and columns for the grid
    batch_size, sequence_length, _ = predictions.shape
    num_rows = int(math.ceil(batch_size / 4))  # 4 columns for a 4x4 grid
    num_cols = 4
    
    # Create a grid of subplots
    fig, axes = plt.subplots(num_rows, num_cols, figsize=(15, 15))
    fig.suptitle("Predictions vs True Values")
    
    # Plot each sequence in the batch
    for i in range(batch_size):
        row = i // num_cols
        col = i % num_cols
        
        ax = axes[row, col] if num_rows > 1 else axes[col]
        
        # Get the predictions and true values for the current sequence
        t = labels[i, :]
        p = predictions[i, :]
        
        # Plot the predictions and true values on the current subplot
        ax.plot(p, label="Predictions")
        ax.plot(t, label="True Values")
        ax.set_title(f"Sequence {i+1}")
        ax.set_ylim(0, 1)

        ax.legend()
        
        
    
    # Adjust layout and display the plot
    plt.tight_layout()
    plt.show()
    break  # Break after processing the first batch

In [ ]:
import torch.nn.functional as F

best_model.eval()
best_model.cpu()

all_predictions = []
all_labels = []

with torch.no_grad():  # Disable gradient calculation to save memory
    for i, (batch_data, batch_labels) in enumerate(val_loader):
        batch_predictions = best_model(batch_data, future=future_steps).cpu()  # Move predictions to CPU
        all_predictions.append(batch_predictions)
        all_labels.append(batch_labels)
        del batch_data, batch_labels, batch_predictions  # Release GPU memory
        torch.cuda.empty_cache()  # Empty the cache to free up memory

all_predictions = torch.cat(all_predictions, dim=0)
all_labels = torch.cat(all_labels, dim=0)

mse = F.mse_loss(all_predictions, all_labels)

print("Mean Squared Error:", mse.item())
